# Demonstração de tradução de texto usando o Azure

Este tutorial apresenta um exemplod de como traduzir um texto usando o *Azure Translator* e o *Azure OpenAI*.

## Instalando os pacotes necessários.

In [1]:
%%capture
!pip install requests python-docx
!pip install beautifulsoup4 openai langchain-openai

## Azure Translator Service

### Definindo a configuração do Azure Translator service

Preencha os parâmetros abaixo com a configuração do seu serviço *Azure Translator*. É necessário criar o serviço anteriormente no site do [Azure](https://azure.microsoft.com) e copiar a chave.

In [2]:
import requests
import os
from docx import Document

# Insert here an API KEY for the Azure Translator service.
subscription_key = "AZURE_TRANSLATOR_KEY"
endpoint = "https://api.cognitive.microsofttranslator.com"
location = "eastus"
target_language ="pt-br"

In [3]:
def translate_text(text, target_language = 'pt-br'):
  path = '/translate'
  constructed_url = endpoint + path

  headers = {
      'Ocp-Apim-Subscription-Key': subscription_key,
      'Ocp-Apim-Subscription-Region': location,
      'Content-type': 'application/json',
      'X-ClientTraceId': str(os.urandom(16))
  }

  body = [{
      'text': text
  }]

  params = {
      'api-version': '3.0',
      'from': 'en',
      'to': target_language
  }

  request = requests.post(constructed_url, params = params, headers=headers, json=body)
  response = request.json()
  return response[0]["translations"][0]["text"]


### Simples teste do tradutor

In [4]:
translate_text("A soul in tension, learning to fly.")

'Uma alma em tensão, aprendendo a voar.'

### Tradução de um documento.

A função abaixo recebe o endereço de um documento em formado Word. Carregue um documento em seu ambiente do Colab para executar o exemplo.

In [5]:
def translate_document(path):
  document = Document(path)
  full_text = []
  for paragraph in document.paragraphs:
    translated_text = translate_text(paragraph.text)
    full_text.append(translated_text)

    translated_doc = Document()
    for line in full_text:
      translated_doc.add_paragraph(line)
      path_translated = path.replace(".docx", f"-{target_language}.docx")
    translated_doc.save(path_translated)
  return full_text

In [6]:
translate_document("/content/learning-fly.docx")

['À distância, uma fita preta',
 'Esticado ao ponto de não voltar atrás',
 'Um vôo de fantasia em um campo varrido pelo vento',
 'De pé sozinho meus sentidos reais',
 'Uma atração fatal me segurando rápido como',
 'Posso escapar desse aperto irresistível?',
 '',
 'Não consigo tirar meus olhos dos céus circulantes',
 'Língua presa e torcida',
 'Apenas um desajustado ligado à terra, eu',
 '',
 'O gelo está se formando nas pontas das minhas asas',
 'Avisos ignorados',
 'Eu pensei que pensei em tudo',
 'Nenhum navegador para encontrar meu caminho de casa',
 'Sem chumbo, vazio e transformado em pedra',
 'Uma alma em tensão que está aprendendo a voar',
 'Condição aterrada, mas determinada a tentar',
 '',
 'Não consigo tirar meus olhos dos céus circulantes',
 'Língua presa e torcida',
 'Apenas um desajustado ligado à terra, eu',
 '',
 'Acima do planeta em uma asa e uma oração',
 'Minha auréola suja, um rastro de vapor no ar vazio',
 'Através das nuvens eu vejo minha sombra voar',
 'Fora do ca

## Azure OpenAI Service

### Carregando o conteúdo de um site.

Esta função carrega o conteudo de uma página web e faz uma limpeza removendo os comandos. O retorno é o texto da página.

In [7]:
import requests
from bs4 import BeautifulSoup

def extract_text_from_url(url):
  response = requests.get(url)

  if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    for script_or_style in soup(["script", "style"]):
      script_or_style.decompose()

    text = soup.get_text(separator=' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text
  else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    return None

extract_text_from_url("https://www.nasa.gov")

"NASA\nExplore\nSearch\nNews & Events\nNews & Events\nAll NASA News\nVideo Series on NASA+\nPodcasts\nBlogs\nNewsletters\nSocial Media\nMedia Resources\nEvents\nUpcoming Launches & Landings\nVirtual Guest Program\nMultimedia\nMultimedia\nNASA+\nImages\nNASA Live\nNASA Apps\nPodcasts\nImage of the Day\ne-Books\nSounds and Ringtones\nInteractives\nSTEM Multimedia\nNASA+\nSearch\nSuggested Searches\nClimate Change\nArtemis\nExpedition 64\nMars perseverance\nSpaceX Crew-2\nInternational Space Station\nView All Topics A-Z\nHome Missions Humans in Space Earth & Climate The Solar System The Universe Science Aeronautics Technology Learning Resources About NASA Español\nNews & Events\nMultimedia\nNASA+\nFeatured 6 min read NASA’s Hubble, Webb Probe Surprisingly Smooth Disk Around Vega article 5 days ago 5 min read NASA, Bhutan Conclude Five Years of Teamwork on STEM, Sustainability article 2 days ago 5 min read What’s Up: November 2024 Skywatching Tips from NASA article 5 days ago\nBack\nMissio

### Configurando o Langchain para acessar o Azure OpenAI Service.

A função abaixo faz a chamada do *Azure OpenAI Service*. Para utilizar é preciso provisionar um serviço no portal Azure e preencher as informações faltantes.

In [8]:
from langchain_openai.chat_models.azure import AzureChatOpenAI

def translate_article(text, target_language = 'português'):
  client = AzureChatOpenAI(
      # Insert your Azure OpenAI service end-point.
      azure_endpoint = 'https://END-POINT.openai.azure.com/',
      # Insert your Azure OpenAI Service key.
      api_key = 'AZURE OPENAI SERVICE KEY',
      api_version = '2024-02-15-preview',
      deployment_name = 'gpt-4o-mini',
      max_retries= 0
  )

  messages = [
      ("system", "Você atua como um  tradutor de textos."),
      ("user", f"Traduza o {text} para o idioma {target_language} e responda em markdown.")
  ]

  client.invoke(messages)

  response = client.invoke(messages)
  print(response.content)
  return response.content

translate_article("A soul in tension, learning to fly.")

```markdown
Uma alma em tensão, aprendendo a voar.
```


'```markdown\nUma alma em tensão, aprendendo a voar.\n```'

### Testando com um site

In [9]:
url = "https://www.nasa.gov"
text = extract_text_from_url(url)
article = translate_article(text)

print(article)

# NASA

## Explorar
- Busca
- Notícias e Eventos
- Todas as Notícias da NASA
- Série de Vídeos no NASA+
- Podcasts
- Blogs
- Newsletters
- Mídias Sociais
- Recursos para Mídia
- Eventos
- Lançamentos e Aterrissagens Futuros
- Programa de Convidados Virtuais
- Multimídia

### Multimídia
- NASA+
- Imagens
- NASA Ao Vivo
- Aplicativos da NASA
- Podcasts
- Imagem do Dia
- e-Books
- Sons e Toques
- Interativos
- Multimídia STEM

## Pesquisa
- Pesquisas Sugeridas
  - Mudanças Climáticas
  - Artemis
  - Expedição 64
  - Perseverança em Marte
  - SpaceX Crew-2
  - Estação Espacial Internacional
- Ver Todos os Tópicos de A a Z

## Início
- Missões
- Humanos no Espaço
- Terra e Clima
- O Sistema Solar
- O Universo
- Ciência
- Aeronáutica
- Tecnologia
- Recursos de Aprendizagem
- Sobre a NASA
- Español

## Destaques
### 6 min de leitura
- [NASA's Hubble, Webb Probe Surprisingly Smooth Disk Around Vega](#) - artigo há 5 dias
### 5 min de leitura
- [NASA, Bhutan Conclude Five Years of Teamwork on S